In [ ]:
# Load packages
import ee
import pandas as pd
from datetime import datetime, date, time, timedelta
from dateutil.relativedelta import *
import geopandas as gpd
import numpy as np
import datatable as dt
from datatable import f
import time
from pprint import pprint
import os, re
from gee_subset import gee_subset
import shapely.geometry
import re
import geemap
from geextract import ts_extract

In [ ]:
# Trigger the authentication flow.
#ee.Authenticate()

# Initialize the library.
ee.Initialize()

In [15]:
country = 'Senegal'

In [17]:
# Import the plot geosjon with the centroids
plots = gpd.read_file("output/plot_data/" + country + "/" + country + "_plot_centroid.GeoJSON")
plots_dt = dt.fread("output/plot_data/" + country + "/" + country + "_plot_centroid.csv")

In [18]:
# Create columns with x and y coordinates
plots['lon'] = plots['geometry'].x
plots['lat'] = plots['geometry'].y 

In [19]:
x_dt = dt.Frame(plots['lon'])
y_dt = dt.Frame(plots['lat'])
plots_dt.cbind(x_dt, y_dt)

In [20]:
# Import the plot geosjon with the polygon geometries
plotsPol = gpd.read_file("output/plot_data/" + country + "/" + country + "_plots_all.GeoJSON")
plotsPol_dt = dt.fread("output/plot_data/" + country + "/" + country + "_plots_all.csv")

In [21]:
# Create ee.Multipoint (for GPM extraction)
geom_list = []
for i in range(x_dt.nrows):
  point = [x_dt[i, 'lon'], y_dt[i, 'lat']]
  geom_list.append(point)

centroid_multi = ee.Geometry.MultiPoint(geom_list)

In [23]:
# Time series settings
start_date = datetime(2022, 10, 27)
end_date = datetime.today()

**Extract L8 polygons**

In [24]:
# Select bands and resolution to extract from landsat 8
#bands = ["SR_B1", "SR_B2", "SR_B3", "SR_B4", "SR_B5", "SR_B6", "SR_B7", "QA_PIXEL", "ST_CDIST", "QA_RADSAT"]
bands = ["SR_B4", "SR_B5", "SR_B6", "QA_PIXEL"]
scale = 30
product='LANDSAT/LC08/C02/T1_L2'

In [25]:
# Create function to get yearly dates for the yearly extraction
def get_year_dates(start, end):
    yearly_dates = [start]
    year_diff = (end.year - start.year) + 1

    for i in range(1, year_diff+1):
        next_year = start_date + relativedelta(years =+ i) 
        yearly_dates.append(next_year)

    return yearly_dates

year_dates = get_year_dates(start_date, end_date)

In [ ]:
refl_l8_pol = dt.Frame()
for pl in range(len(plotsPol)):
    print(pl)

    g = [i for i in plotsPol.geometry]
    x,y = g[pl].exterior.coords.xy
    cords = np.dstack((x,y)).tolist()
    geometry = ee.Geometry.Polygon(cords)
    geometry = geometry.buffer(0.000063)

    # Check if geometry exceeds critical size (15ha)
    if plotsPol['Hectare'].iloc[pl] > 100:
        print('Area too large --> extracting per year')
        # Extract data per year to prevent GEE size error
        for yr in range(len(year_dates)-1):
            try:
                col = ee.ImageCollection(product).\
                    select(tuple(bands)).\
                    filterDate(year_dates[yr], year_dates[yr+1]).filterBounds(geometry)


                # Make a df
                region = col.getRegion(geometry, int(scale)).getInfo()
                df = pd.DataFrame.from_records(region[1:len(region)])
                df.columns = region[0]
                df = df[['time', 'SR_B4', 'SR_B5', 'SR_B6', 'QA_PIXEL']]
  
                df.time = df.time / 1000
                df['time'] = pd.to_datetime(df['time'], unit = 's')
                df.rename(columns = {'time': 'date'}, inplace = True)
                df.sort_values(by = 'date')

                # Transform to dt
                l8_out = dt.Frame(df)

                # Create column with plotID 
                l8_out['plotID'] = plotsPol['plotID'].iloc[pl]
                refl_l8_pol.rbind(l8_out)

            except:
                pass
                
    else:
        col = ee.ImageCollection(product).\
            select(tuple(bands)).\
            filterDate(start_date, end_date).filterBounds(geometry)

        region = col.getRegion(geometry, int(scale)).getInfo()

        # If no pixels in geometry, take centroid of plot
        if len(region) == 1:
            print('Not enough pixels in geometry (taking centroid)')
            
            geometry = ee.Geometry.Point([plots_dt[pl,'lon'], plots_dt[pl, 'lat']])
            col = ee.ImageCollection(product).\
                select(tuple(bands)).\
                filterDate(start_date, end_date).filterBounds(geometry)
      
        region = col.getRegion(geometry, int(scale)).getInfo()
        df = pd.DataFrame.from_records(region[1:len(region)])
        df.columns = region[0]
        df = df[['time', 'SR_B4', 'SR_B5', 'SR_B6', 'QA_PIXEL']]
  
        df.time = df.time / 1000
        df['time'] = pd.to_datetime(df['time'], unit = 's')
        df.rename(columns = {'time': 'date'}, inplace = True)
        df.sort_values(by = 'date')

        # Transform to dt
        l8_out = dt.Frame(df)

        # Create column with plotID 
        l8_out['plotID'] = plotsPol['plotID'].iloc[pl]

        refl_l8_pol.rbind(l8_out)

In [27]:
refl_l8_pol.to_csv("output/gee/" + country + "_refl_l8_pol_final_months.csv")

In [53]:
refl_l8_pol = dt.fread("output/gee/Ethiopia_refl_l8_pol.csv")

**Extract Precipitation data (GPM)**

In [11]:
## Set parameters for the time series
start_ts = datetime(2013, 1, 1)
end_ts = datetime.today()

# Specify number of days in period of interest
d0 = datetime(start_ts.year, start_ts.month, start_ts.day)
d1 = datetime(end_ts.year, end_ts.month, 1)
delta = d1 - d0
days = delta.days

# number of months in period
def diff_month(d1, d2):
    return (d1.year - d2.year) * 12 + d1.month - d2.month
months_ts = diff_month(d1, d0)

# Create list with the dates off all the observations
months_date = []
for m in range(months_ts):
  first_month = start_ts
  next_month = first_month + relativedelta(months =+ m)
  months_date.append(next_month)

In [12]:
# Extract all the precipitation data for all sites
gpm = ee.ImageCollection('NASA/GPM_L3/IMERG_V06').\
       select('precipitationCal').\
       filterDate(start_ts, end_ts).filterBounds(centroid_multi)

In [13]:
# Create a function to go over the FeatureCollection and take the monthly sum
def GPMsum(img_collection):
  mylist = ee.List([])
  for m in range(months_ts):

    ini = start_ts + relativedelta(months=+m)
    end = ini + relativedelta(months=+1) + relativedelta(days=-1)

    sum_image = img_collection.filterDate(ini,end).select(0).sum()
    mylist = mylist.add(sum_image.set('system:time_start', ini))
  return ee.ImageCollection.fromImages(mylist)

In [14]:
# Apply the 'GPMsum' function to create FeatureCollection with monthly sum 
monthlyGPM = ee.ImageCollection(GPMsum(gpm))
# Sort FeatureCollection by date and create single image with bands
monthlyGPM_stack = monthlyGPM.sort('system:time_start').toBands().multiply(0.5)

In [15]:
img_todrive = {
    'description': country + '_GPM_stack',
    'folder': 'Regreening_Africa',
    'scale': 11000,
    'maxPixels': 1e13,
    'region': centroid_multi,
    'fileFormat': 'GeoTIFF'}

task = ee.batch.Export.image.toDrive(monthlyGPM_stack, **img_todrive)
task.start()

In [16]:
task.status()

{'state': 'READY',
 'description': 'Ethiopia_GPM_stack',
 'creation_timestamp_ms': 1669124103484,
 'update_timestamp_ms': 1669124103484,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_IMAGE',
 'id': 'ULRRZWXHYE5TC25DZOO7MNPM',
 'name': 'projects/earthengine-legacy/operations/ULRRZWXHYE5TC25DZOO7MNPM'}